This notebook loops through the local experiment folder and removes the large files/variables associated with each experiment and stores a single copy of them in a separate file

In particular, the backtest results pkls have a lot of extra information that is not required

In [1]:
import pandas as pd
from services.experiments import *
import matplotlib.pyplot as plt

In [4]:
universe = 'ETF'

CardMVO_path = "Experiments//"+universe+"//"+"CardMVO"
CardMVO_experiments = pd.read_pickle(CardMVO_path+"//data_dictionary.pkl")

# add filters here


#assert (CardMVO_experiments.groupby(['cardinality_ratio', 'turnover_limit']).uid.count() == 1).all()

unique_card_turnover_df = CardMVO_experiments.uid #.first() # unique experiments

#def read_results(CardMVO_experiments):
estimators = CardMVO_experiments.groupby(['estimator', 'EstNumObs', 'k', 'alpha']).first()
estimator_idx = 0

contexts_by_estimator = {}
mu_by_estimator = {}
cov_by_estimator = {}

for uid in unique_card_turnover_df:

    results = import_dict(CardMVO_path, uid, 'backtest_results')

    contexts_by_period = {} #indexed by t; period context, and cov
    mu_by_period = {}
    cov_by_period = {}

    if uid == estimators.iloc[estimator_idx].uid: #on the first uid save stuff
        estimator_key = estimators.index[estimator_idx]
        for t in results.keys():
            period_results = results[t]
            if 'optimization_params' in period_results.keys():
                if 'period_Context' in period_results['optimization_params'].keys():
                    contexts_by_period[t] = period_results['optimization_params']['period_Context']

            if 'mu' in period_results.keys():
                mu_by_period[t] = period_results['mu']

            if 'cov' in period_results.keys():
                cov_by_period[t] = period_results['cov']

        contexts_by_estimator[estimator_key] = contexts_by_period
        mu_by_estimator[estimator_key] = mu_by_period
        cov_by_estimator[estimator_key] = cov_by_period

        estimator_idx = min(len(estimators)-1, estimator_idx+1) #move on to the next estimator

    delete_flag = False
    for t in results.keys():
        period_results = results[t]
        if 'optimization_params' in period_results.keys():
            if 'period_Context' in period_results['optimization_params'].keys():
                del results[t]['optimization_params']['period_Context']
                delete_flag = True
            if 'mu' in period_results['optimization_params'].keys():
                del results[t]['optimization_params']['mu']
                delete_flag = True

            if 'Q' in period_results['optimization_params'].keys():
                del results[t]['optimization_params']['Q']
                delete_flag = True

        if 'mu' in period_results.keys():
            del results[t]['mu']
            delete_flag = True

        if 'cov' in period_results.keys():
            del results[t]['cov']
            delete_flag = True


            # exiting loop over all experiments
        export_dict(CardMVO_path, uid, results, 'backtest_results')

if delete_flag:
    # exiting loop over all experiments
    print("Deleting")
    export_dict(CardMVO_path, '', contexts_by_estimator, 'contexts_by_estimator')
    export_dict(CardMVO_path, '', mu_by_estimator, 'mu_by_estimator')
    export_dict(CardMVO_path, '', cov_by_estimator, 'cov_by_estimator')
else:
    print("No changes made")

Deleting


In [6]:

SVMMVO_path = "Experiments//"+universe+"//"+"SVMMVO"
qs = [3]


for q in qs:
    SVMMVO_path_q = SVMMVO_path+"//"+str(q)
    SVMMVO_experiments_q = pd.read_pickle(SVMMVO_path_q+"//data_dictionary.pkl")

    estimators = SVMMVO_experiments_q.groupby(['estimator', 'EstNumObs', 'k', 'alpha']).first()
    estimator_idx = 0
    print(estimators)

    contexts_by_estimator = {}
    mu_by_estimator = {}
    cov_by_estimator = {}

    print("q =", q, " runs available: ", SVMMVO_experiments_q.run.unique())

    assert len(SVMMVO_experiments_q.uid) == len(SVMMVO_experiments_q.uid.unique())

    for uid in SVMMVO_experiments_q.uid:
        results = import_dict(SVMMVO_path_q, uid, 'backtest_results')

        contexts_by_period = {} #indexed by t; period context, and cov
        mu_by_period = {}
        cov_by_period = {}

        # new estimator encountered
        # store
        if uid == estimators.iloc[estimator_idx].uid: #on the first uid save stuff
            estimator_key = estimators.index[estimator_idx]
            for t in results.keys():
                period_results = results[t]
                if 'optimization_params' in period_results.keys():
                    if 'period_Context' in period_results['optimization_params'].keys():
                        contexts_by_period[t] = period_results['optimization_params']['period_Context']

                if 'mu' in period_results.keys():
                    mu_by_period[t] = period_results['mu']

                if 'cov' in period_results.keys():
                    cov_by_period[t] = period_results['cov']

            contexts_by_estimator[estimator_key] = contexts_by_period
            mu_by_estimator[estimator_key] = mu_by_period
            cov_by_estimator[estimator_key] = cov_by_period

            estimator_idx = min(len(estimators)-1, estimator_idx+1) #move on to the next estimator
        delete_flag = False
        for t in results.keys():
            period_results = results[t]
            if 'optimization_params' in period_results.keys():
                if 'period_Context' in period_results['optimization_params'].keys():
                    del results[t]['optimization_params']['period_Context']
                    delete_flag = True
                if 'mu' in period_results['optimization_params'].keys():
                    del results[t]['optimization_params']['mu']
                    delete_flag = True
                if 'Q' in period_results['optimization_params'].keys():
                    del results[t]['optimization_params']['Q']
                    delete_flag = True
            if 'mu' in period_results.keys():
                del results[t]['mu']
                delete_flag = True
            if 'cov' in period_results.keys():
                del results[t]['cov']
                delete_flag = True

            # exiting loop over all experiments
        export_dict(SVMMVO_path_q, uid, results, 'backtest_results')

    if delete_flag:
    # exiting loop over all experiments
        print("Deletion made")
        export_dict(SVMMVO_path_q, '', contexts_by_estimator, 'contexts_by_estimator')
        export_dict(SVMMVO_path_q, '', mu_by_estimator, 'mu_by_estimator')
        export_dict(SVMMVO_path_q, '', cov_by_estimator, 'cov_by_estimator')
        print("estimators ", contexts_by_estimator.keys())
    else:
        print("No changes made")
    # delete the unnesssary files

                                                                                                   uid  \
estimator                                EstNumObs k   alpha                                             
exponential_weighted_estimator           750       120 0.01   0dee79e2b6965a0d8531d29884077bb8ce850702   
exponential_weighted_estimator_shrinkage 750       120 0.01   6b632091ebf7c2346305bdf2f7252840f9b1f27a   

                                                             optimizer  \
estimator                                EstNumObs k   alpha             
exponential_weighted_estimator           750       120 0.01     SVMMVO   
exponential_weighted_estimator_shrinkage 750       120 0.01     SVMMVO   

                                                             universe  \
estimator                                EstNumObs k   alpha            
exponential_weighted_estimator           750       120 0.01       ETF   
exponential_weighted_estimator_shrinkage 750       120 0.01